In [11]:
import cv2
import os
from datetime import datetime
import time
import socket
import subprocess
import Helper
import pandas as pd
from super_gradients.training import models
import numpy as np


In [12]:
import socket
def get_ip_address():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #creat connection
        s.connect(("8.8.8.8", 80)) #ping google
        ip = s.getsockname()[0] #own ip add.
        s.close()
        return ip
    except Exception as e:
        return F"Could not get IP: {e}"

get_ip_address()

'172.20.10.4'

## Calling Model


In [13]:
#Download Model from model link.txt file
model_path = r"D:\Final Code\Final Project\Yolo nas\Dataset\checkpoints\tips and liquid\RUN_20240120_202748_124739\ckpt_best.pth"

best_model = models.get('yolo_nas_l',
                        num_classes=2,
                        checkpoint_path=model_path)

[2024-03-05 20:30:10] INFO - checkpoint_utils.py - Successfully loaded model weights from D:\Final Code\Final Project\Yolo nas\Dataset\checkpoints\tips and liquid\RUN_20240120_202748_124739\ckpt_best.pth EMA checkpoint.


In [11]:
def start_server():
    keys = ['Tip1', 'Tip2', 'Tip3', 'Tip4', 'Tip5', 'Tip6', 'Tip7', 'Tip8']
    Tips_check = {key: [] for key in keys}
    Liquid_check = {key: [] for key in keys}
    Tips_volume = {key: [] for key in keys}

    keys2 = ['Tips_Count', 'Liquid_Count']
    counts_data = {key: [] for key in keys2}

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    port = 9999
    server_socket.bind(('', port))
    server_socket.listen(1)
    print(f"Server started on port {port}")

    cap = Helper.initialize_camera()
    project_name='default_project'

    try:
        while True:  #Loop to accept new connections
            client_socket, addr = server_socket.accept()
            print(f"Got a connection from {addr}")
            while True:  #Loop to handle commands from a connected client
                command = client_socket.recv(1024).decode('utf-8').strip().lower()
                if not command:  # Break the loop if the command is empty (client disconnected)
                    break
                if command == 'capture_image':
                    #response = Helper.capture_hd_image_with_lock('default_project')
                    response = Helper.capture_live_image(cap, project_name)
                    client_socket.send(response.encode('utf-8'))

                    predictions = best_model.predict([response], conf=0.7)
                    Helper.save_predictions(predictions, output_folder="output_predictions")

                    class_counts, bounding_boxes, bounding_box_centers, liquid_height_percentages = Helper.process_predictions(predictions)
                    tip_presence, missing_tip_positions = Helper.find_missing_tips(bounding_box_centers)

                    liquid_values = liquid_height_percentages['Liquid']
                    tip_index = 0

                    #Iterate through the 'Liquid' values and assign to 'Tips_volume' based on 'tip_presence'
                    for i, presence in enumerate(tip_presence):
                        tip_key = f'Tip{i + 1}'
                        if presence == 1:
                            Tips_volume[tip_key].append(liquid_values[tip_index])
                            tip_index += 1
                        else:
                            Tips_volume[tip_key].append(0)

                    counts_data['Tips_Count'].append(class_counts['Tip'])
                    counts_data['Liquid_Count'].append(class_counts['Liquid'])

                    for i, presence in enumerate(tip_presence):
                        tip_key = f'Tip{i + 1}'
                        if presence == 1:
                            Tips_check[tip_key].append("Attached")
                        else:
                            Tips_check[tip_key].append("MISSING")

                elif command == 'shutdown_server':
                    print("Shutting down server.")
                    server_socket.close()
                    return Tips_check, Tips_volume, counts_data
                    Helper.release_camera(cap)
                else:
                    print(f"Unknown command: {command}")
    except Exception as e:
        print(f"Server error: {e}")
    finally:
        server_socket.close()
        print("Server has been shut down.")
        Helper.release_camera(cap)

if __name__ == "__main__":
    Tips_check, Tips_volume, counts_data = start_server()
    Helper.release_camera(cap)

Server started on port 9999
Got a connection from ('172.20.10.6', 37612)


In [3]:
Tips_volume 
Tips_volume1 = pd.DataFrame(Tips_volume)
Tips_volume1.index = [f'ROW {i + 1}' for i in range(len(Tips_volume1))]
Tips_volume1=Tips_volume1.round()
Tips_volume1

In [4]:
Tips_check = pd.DataFrame(Tips_check)
Tips_check.index = [f'ROW {i + 1}' for i in range(len(Tips_check))]
Tips_check

Camera released


In [ ]:
counts_data = pd.DataFrame(counts_data)
counts_data.index = [f'Round {i + 1}' for i in range(len(counts_data))]
counts_data

In [ ]:
writer = pd.ExcelWriter('dataframes_combined.xlsx', engine='xlsxwriter')
Tips_volume1.to_excel(writer, sheet_name='Tips_volume1')
Tips_check.to_excel(writer, sheet_name='Tips_check')
counts_data.to_excel(writer, sheet_name='counts_data')
writer.close()
print("Logs Saved")

In [29]:
#TEST
project_name = "one"
cap = Helper.initialize_camera()


In [26]:
Helper.capture_image(cap, project_name)

'd:\\Final Code\\Final Project\\Demo codes\\one\\one_20240305193938_hd.jpg'

Image captured with locked focus: d:\Final Code\Final Project\Demo codes\one\one_20240305193938_hd.jpg


In [36]:
Helper.release_camera(cap)

Camera released


In [31]:
def capture_image(cap, project_name):
    #Read the frame from the camera
    ret, frame = cap.read()

    #Check if frame is successfully captured
    if not ret:
        print("Failed to capture image")
        return None

    #Create unique image path for each capture
    image_path = Helper.create_project(project_name)

    #Save the image to the specified path
    cv2.imwrite(image_path, frame)
    print(f'Image captured with locked focus: {image_path}')

    return image_path


In [35]:
project_name = "one" #change to yoru project name like PCR or anything.
image_path = capture_image(cap, project_name)

Image captured with locked focus: d:\Final Code\Final Project\Demo codes\one\one_20240305194704_hd.jpg


In [62]:
# GPT Below


In [6]:
import cv2
import os
from datetime import datetime

def create_project(project_name = 'default_project'):
    # Join the current directory with the project name to create the project folder path
    project_folder = os.path.join(os.path.abspath('.'), project_name)
    # Create the project folder if it doesn't exist
    os.makedirs(project_folder, exist_ok=True)
    # Get the current time and format it
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    # Create the image name using project name and current time
    image_name = f'{project_name}_{current_time}_hd.jpg'
    # Join the project folder with the image name to get the full image path
    image_path = os.path.join(project_folder, image_name)
    # Return the image path
    return image_path

def initialize_camera():
    # Initialize the camera
    cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)  #'0' is typically the default camera ID. mine has 3 cams attached so its 1 for me.
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  #Set the width
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  #Set the height
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))  # Set the codec
    cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)  #Turn off autofocus

    #Set focus value for closer objects (adjust as needed)
    focus_value = 50.0  #Change this value to your desired focus setting mine is best at 50mm.
    cap.set(cv2.CAP_PROP_FOCUS, focus_value)  # Set the focus property

    return cap

def capture_live_image(cap, project_name):
    #Discard any previously read frames
    for _ in range(10):  # Read and discard 10 frames
        cap.read()

    #Read the latest frame from the camera
    ret, frame = cap.read()

    #Check if frame is successfully captured
    if not ret:
        print("Failed to capture image")
        return None

    #Create unique image path for each capture
    image_path = create_project(project_name)

    #Save the image to the specified path
    cv2.imwrite(image_path, frame)
    print(f'Image captured with locked focus: {image_path}')

    return image_path

def release_camera(cap):
    #Release the camera
    cap.release()
    print("Camera released")

#Example usage:
project_name = "one"

#Initialize the camera
cap = initialize_camera()



In [8]:
capture_live_image(cap, project_name)

Image captured with locked focus: d:\Final Code\Final Project\Demo codes\one\one_20240305202249_hd.jpg


'd:\\Final Code\\Final Project\\Demo codes\\one\\one_20240305202249_hd.jpg'

In [9]:
release_camera(cap)

Camera released
